In [1]:
from _ast import *
import ast

In [4]:
from os import listdir
from os.path import isfile, join, isdir

def get_all_functions_path(module_name, path):
    function_dict = {}
    class_dict = {}
    prefix = module_name + "."
    for element in sorted(listdir(path)):
        if isfile(join(path, element)):
            sub_module_name = element.split(".py")[0]
            source = open(join(path, element), "r").read()
            tree = ast.parse(source)
            function_list = []
            class_list = []
            for element in tree.body:
                if type(element) == FunctionDef and element.name[0] != "_":
                    function_list.append(element)
                elif type(element) == ClassDef and element.name[0] != "_":
                    class_list.append(element)
            if len(function_list) > 0:
                function_dict[prefix + sub_module_name] = function_list
            if len(class_list) > 0:
                class_dict[prefix + sub_module_name] = class_list
        elif isdir(join(path, element)):
            res_function_dict, res_class_dict = get_all_functions_path(prefix + element, join(path, element))
            function_dict.update(res_function_dict)
            class_dict.update(res_class_dict)
    return function_dict, class_dict
    
function_dict, class_dict = get_all_functions_path("calculator", "crawl/packages/calculator-0.0.1/calculator")

In [8]:
def get_class_functions(class_dict):
    class_function_dict = {}
    for module in class_dict.keys():
        class_function_list = []
        for class_def in class_dict[module]:
            for element in class_def.body:
                if type(element) == FunctionDef and element.name[0] != "_":
                    class_function_list.append({
                        "dependend_object": class_def.name,
                        "function": element
                    })
        class_function_dict[module] = class_function_list
    return class_function_dict
class_function_dict = get_class_functions(class_dict)

In [56]:
source = open("crawl/packages/calculator-0.0.1/calculator/list_functions.py", "r").read()
tree = ast.parse(source)

for node in ast.walk(tree.body[2].body[1]):
    if type(node) == Call:
        trace_node = node.func
        function_trace = trace_node.attr
        while True:
            try:
                trace_node = trace_node.value
                function_trace = "{}.".format(trace_node.attr) + function_trace
            except:
                function_trace = "{}.".format(trace_node.id) + function_trace
                break
        print(function_trace)

self.list.append


In [63]:
import builtins

source = open("crawl/packages/calculator-0.0.1/calculator/base_functions.py", "r").read()
tree = ast.parse(source)

for node in ast.walk(tree.body[2]):
    if type(node) == Call:
        trace_node = node.func
        if type(trace_node) == Attribute:
            function_trace = trace_node.attr
            while True:
                try:
                    trace_node = trace_node.value
                    function_trace = "{}.".format(trace_node.attr) + function_trace
                except:
                    function_trace = "{}.".format(trace_node.id) + function_trace
                    break
            print(function_trace)
        else:
            if trace_node.id not in dir(builtins):
                print(trace_node.id)

add
